In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [3]:
# Define the model architecture
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [4]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [5]:
# Set up data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    classes=['class0', 'class1'])

Found 455 images belonging to 2 classes.


In [7]:
validation_generator = test_datagen.flow_from_directory(
    'dataset/test',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    classes=['class0', 'class1'])

Found 64 images belonging to 2 classes.


In [14]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator))

# Save the model
model.save('glaucoma_detection_model.h5')


Epoch 1/10
15/15 [==============================] - 38s 2s/step - loss: 1.0893 - accuracy: 0.5868 - val_loss: 0.6961 - val_accuracy: 0.5000
Epoch 2/10
15/15 [==============================] - 30s 2s/step - loss: 0.6766 - accuracy: 0.5429 - val_loss: 0.6353 - val_accuracy: 0.6719
Epoch 3/10
15/15 [==============================] - 31s 2s/step - loss: 0.5311 - accuracy: 0.7187 - val_loss: 0.5700 - val_accuracy: 0.6719
Epoch 4/10
15/15 [==============================] - 28s 2s/step - loss: 0.4306 - accuracy: 0.7978 - val_loss: 0.5172 - val_accuracy: 0.7344
Epoch 5/10
15/15 [==============================] - 28s 2s/step - loss: 0.3902 - accuracy: 0.8330 - val_loss: 0.4199 - val_accuracy: 0.8281
Epoch 6/10
15/15 [==============================] - 29s 2s/step - loss: 0.3734 - accuracy: 0.8418 - val_loss: 0.5905 - val_accuracy: 0.6875
Epoch 7/10
15/15 [==============================] - 27s 2s/step - loss: 0.3335 - accuracy: 0.8615 - val_loss: 0.3738 - val_accuracy: 0.8438
Epoch 8/10
15/15 [==

In [8]:
# Evaluate the model on the validation set
loaded_model = tf.keras.models.load_model('glaucoma_detection_model.h5')
loss, accuracy = loaded_model.evaluate(validation_generator)
print('Validation loss:', loss)
print('Validation accuracy:', accuracy)

2/2 [==============================] - 17s 640ms/step - loss: 0.2554 - accuracy: 0.9219
Validation loss: 0.25540608167648315
Validation accuracy: 0.921875


In [9]:
from keras.models import load_model
from PIL import Image
from keras.preprocessing import image
import numpy as np
import cv2

In [10]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('dataset/test/class1/Im263.jpg', target_size = (256,256))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = loaded_model.predict(test_image)
#train_generator.class_indices
if result == 1:
 print("Glaucoma")
else:
 print("Not Glaucoma")

1/1 [==============================] - 0s 425ms/step
Glaucoma


In [11]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('Fundus_Scanes_Sorted/Validation/Glaucoma_Positive/621.jpg', target_size = (256,256))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = loaded_model.predict(test_image)
#train_generator.class_indices
if result[0][0] == 1:
 print("Glaucoma")
else:
 print("Not Glaucoma")

1/1 [==============================] - 0s 50ms/step
Not Glaucoma


## Using VGG16

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [4]:
from tensorflow.keras.applications import VGG16

# Define the model architecture
model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))
model.trainable = False
x = keras.layers.Flatten()(model.output)
x = keras.layers.Dense(128, activation='relu')(x)
predictions = keras.layers.Dense(1, activation='sigmoid')(x)
model = keras.models.Model(inputs=model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Set up data generators
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    classes=['class0', 'class1'])

validation_generator = test_datagen.flow_from_directory(
    'dataset/test',
    target_size=(256, 256),
    batch_size=32,
    class_mode='binary',
    classes=['class0', 'class1'])

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=30,
    validation_data=validation_generator,
    validation_steps=len(validation_generator))

# Save the model
model.save('glaucoma_detection_model_vgg16.h5')


Found 455 images belonging to 2 classes.
Found 64 images belonging to 2 classes.
Epoch 1/30
15/15 [==============================] - 157s 10s/step - loss: 1.5590 - accuracy: 0.5516 - val_loss: 0.6774 - val_accuracy: 0.7188
Epoch 2/30
15/15 [==============================] - 146s 10s/step - loss: 0.6349 - accuracy: 0.6088 - val_loss: 0.6299 - val_accuracy: 0.6719
Epoch 3/30
15/15 [==============================] - 154s 11s/step - loss: 0.6097 - accuracy: 0.6857 - val_loss: 0.6078 - val_accuracy: 0.7031
Epoch 4/30
15/15 [==============================] - 165s 11s/step - loss: 0.5873 - accuracy: 0.7165 - val_loss: 0.5930 - val_accuracy: 0.7500
Epoch 5/30
15/15 [==============================] - 160s 11s/step - loss: 0.5692 - accuracy: 0.6967 - val_loss: 0.5992 - val_accuracy: 0.8125
Epoch 6/30
15/15 [==============================] - 159s 11s/step - loss: 0.5623 - accuracy: 0.7187 - val_loss: 0.5696 - val_accuracy: 0.7969
Epoch 7/30
15/15 [==============================] - 170s 11s/step -

In [5]:
# Evaluate the model on the validation set
loss, accuracy = model.evaluate(validation_generator)
print('Validation loss:', loss)
print('Validation accuracy:', accuracy)

2/2 [==============================] - 21s 10s/step - loss: 0.3611 - accuracy: 0.9531
Validation loss: 0.3610827922821045
Validation accuracy: 0.953125


In [6]:
from keras.models import load_model
from PIL import Image
from keras.preprocessing import image
import numpy as np
import cv2